In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from functools import partial


# Python y rendimiento | Parte 3


1. Aprovechando arquitecturas multi-nucleo con `Multiprocessing`




In [ ]:
from multiprocessing import Process

def foo():
    print('hello')

p = Process(target=foo)
p.start()

https://github.com/mynameisfiber/high_performance_python

https://joblib.readthedocs.io/en/latest/parallel.html

In [ ]:
from math import sin, exp
from joblib import Parallel, delayed


In [ ]:
%%time
N = 100000000
a = [sum([1]*N) for i in range(4)]

In [ ]:
%%time 
b = Parallel(n_jobs=4)(delayed(lambda x: sum([1]*N))(i) for i in range(4))

In [ ]:
np.allclose(a, b)

# Computación paralela con IPython: [ipyparallel](https://ipyparallel.readthedocs.io/en/latest/)


https://ipython-books.github.io/59-distributing-python-code-across-multiple-cores-with-ipython/

Iniciamos un cluster de cuatro procesos con

    ipcluster start -n 4
    
o desde la pestaña "IPython clusters" del jupyter notebook

In [ ]:
import ipyparallel as ipp

c = ipp.Client()

display(c.ids)

c[:].apply_sync(lambda : "Hello, World")

## Global Interpreter Lock (GIL)

El intérprete CPython no es *thread-safe*

Es por esto que en Python existe el *Global Interpreter Lock* (GIL)

> El GIL solo permite que solo un hilo a la vez pueda ejecutar código en Python

Usualmente un proceso Python no puedo usar múltiples CPU

El código escrito en Python no tiene control sobre el GIL

> Pero, extensiones escritas en C o Cython si pueden liberar GIL

## Liberando el GIL en Cython

https://ipython-books.github.io/57-releasing-the-gil-to-take-advantage-of-multi-core-processors-with-cython-and-openmp/

- Multiprocessing

https://pjryan126.github.io/multiprocessing-in-python/

https://github.com/jupyter/notebook/issues/1703

https://medium.com/@grvsinghal/speed-up-your-code-using-multiprocessing-in-python-36e4e703213e

https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac

# Computación de alto rendimiento

Ciertos problemas son tan extensos que para solucionarlos en un tiempo razonable se requiere poder de cómputo y/o memoria superior a la ofrecida por un computador de arquitectura tradicional

Si el problema es separable entonces puede resolverse de forma eficiente usando computación paralela y/o computación distributida

La computación de alto rendimiento o *high-performance computing* (HPC) es la disciplina que se dedica a diseñar algoritmos eficientes que utilizan arquitecturas paralelas/distribuidas

En HPC también se investiga la utilización eficiente de co-procesadores de alto paralelismo como son los procesadores de tipo *many-core* (Xeon-Phi) y las Graphical Processing Units (GPU, Nvidia, AMD)

El Magíster en Informática de la UACh tiene a HPC como una de sus áreas principales

Más información en: http://www.ingenieria.uach.cl/index.php/postgrado/magister-en-informatica



In [ ]:
import pycuda.driver as drv
import pycuda.tools
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
from pycuda.elementwise import ElementwiseKernel

complex_gpu = ElementwiseKernel(
    "pycuda::complex<float> *q, int *output, int maxiter",
    """
    {
        float nreal, real = 0;
        float imag = 0;
        output[i] = 0;
        for(int curiter = 0; curiter < maxiter; curiter++) {
            float real2 = real*real;
            float imag2 = imag*imag;
            nreal = real2 - imag2 + q[i].real();
            imag = 2* real*imag + q[i].imag();
            real = nreal;
            if (real2 + imag2 > 4.0f){
                output[i] = curiter;
                break;
                };
        };
    }
    """,
    "complex5",
    preamble="#include <pycuda-complex.hpp>",)

def mandelbrot_gpu(c, maxiter):
    q_gpu = gpuarray.to_gpu(c.astype(np.complex64))
    iterations_gpu = gpuarray.to_gpu(np.empty(c.shape, dtype=np.int))
    complex_gpu(q_gpu, iterations_gpu, maxiter)

    return iterations_gpu.get()